In [13]:
# Import
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

import os
# Create SparkSession
spark = SparkSession.builder.appName("Bankstatementsprocessing").getOrCreate()


In [14]:
from datetime import datetime

# Get today's date
today_date = datetime.today()

# Convert the date to a string in the format "YYYY-MM-DD"
today_date_str = today_date.strftime("%Y-%m-%d")
# Folder path containing CSV files
folder_path = "C:/Projects/Finances/Bank statements"
folders = os.listdir(folder_path)
for folder in folders:
    try:
        # List CSV files in the folder
        csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

        # Read CSV files into Spark DataFrame
        df = spark.read.csv(csv_files , header=True , inferSchema=True )

        # The desired columns and order
        columns = ['Date', 'Description', 'Debit', 'Credit', 'Balance']

        # Select the desired columns in the specified order
        df = df.select(*[col(column_name) for column_name in columns])

        output_file_path = f"{folder_path}/{folder}_consolidated_{today_date_str}.parquet"
        # Write the DataFrame to a Parquet file with the specified file name
        df.write.parquet(output_file_path)
    except Exception as e:
        print( f"Error occured while processing {folder} :: {str(e)}")
    

AnalysisException: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for CSV. It must be specified manually.

In [15]:
folder_path = "C:/Projects/Finances/Bank statements/au"

In [16]:
# List CSV files in the folder
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# Read CSV files into Spark DataFrame
df = spark.read.csv(csv_files , header=True , inferSchema=True )

# The desired columns and order
columns = ['Date', 'Description', 'Debit', 'Credit', 'Balance']

# Select the desired columns in the specified order
df = df.select(*[col(column_name) for column_name in columns])

In [19]:
df.count()

165

'C:/Projects/Finances/Bank statements/hdfc_consolidated_2024-05-14.parquet'

In [2]:

for 
# Folder path containing CSV files
folder_path = "C:/Projects/Finances/Bank statements/hdfc"




In [ ]:
# Specify the output folder path
output_folder = "C:/Projects/Finances/Bank statements/"

# Save DataFrame as Parquet files in the specified folder
df.write.parquet(output_folder)

In [101]:
df.createOrReplaceTempView("hdfc")

In [102]:
sql = """Select *
                      
         from hdfc
         order by date
         """

In [107]:
#spark.sql(sql).show()

In [89]:
#df.show()

In [108]:
#df.show()

df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Debit: double (nullable = true)
 |-- Credit: double (nullable = true)
 |-- Balance: double (nullable = true)



In [100]:
from pyspark.sql.functions import to_date, when

df = df.withColumn('Date', when(to_date(df["Date"], "dd/MM/yy").isNull(), None) \
                   .otherwise(to_date(df["Date"], "dd/MM/yy")))

In [112]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
from datetime import datetime

# Define a Python function to parse date strings
def parse_date(date_str):
    try:
        return datetime.strptime(date_str, '%d/%m/%y').date()
    except:
        return None

# Register the function as a UDF
parse_date_udf = udf(parse_date, DateType())

# Apply the UDF to the 'Date' column
df = df.withColumn('Date', parse_date_udf(df['Date']))

# Show the DataFrame
df.show(2)


Py4JJavaError: An error occurred while calling o677.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 73.0 failed 1 times, most recent failure: Lost task 0.0 in stage 73.0 (TID 129) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:82)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:853)
	at org.apache.spark.rdd.RDD.$anonfu

In [34]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Description: string (nullable = true)
 |-- Value_date: string (nullable = true)
 |-- Debit: double (nullable = true)
 |-- Credit: double (nullable = true)
 |-- Ref_No: string (nullable = true)
 |-- Balance: double (nullable = true)



Checking metadata and viewing the rows where issue is coming

In [12]:
# Get the schema as a StructType object
schema = df.schema

# Get the metadata for a specific column (replace "column_name" with the actual column name)
column_metadata = schema["Debit"]

# Print the metadata to see how the data type was inferred
print("Column Metadata:", column_metadata)

Column Metadata: StructField('Debit', StringType(), True)


In [21]:
from pyspark.sql.functions import col

# Convert column to numeric (assuming the column name is "Debit")
df = df.withColumn("Debit", df["Debit"].cast("double"))

# Find NA values in the column
na_count = df.filter(col("Debit").isNull()).count()

# Show DataFrame schema to confirm the data type conversion
df.printSchema()

# Print the count of NA values
print("Number of NA values in 'Debit':", na_count)


root
 |-- Date: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Value_date: string (nullable = true)
 |-- Debit: double (nullable = true)
 |-- Credit: double (nullable = true)
 |-- Ref_No: string (nullable = true)
 |-- Balance: double (nullable = true)

Number of NA values in 'Debit': 0


In [44]:
df.filter(col("Date").isNull()).show()

+----+-----------+-----+------+-------+
|Date|Description|Debit|Credit|Balance|
+----+-----------+-----+------+-------+
+----+-----------+-----+------+-------+



In [72]:
from pyspark.sql.functions import udf
import re

def validate_date_format(date_str):
  """
  Checks if the input string matches the 'dd/MM/yy' format.
  Args:
      date_str (str): The date string to validate.
  Returns:
      bool: True if the format is valid, False otherwise.
  """
  pattern = r"^\d{2}/\d{2}/\d{2}$"
  return bool(re.match(pattern, date_str))

validate_udf = udf(lambda x: validate_date_format(x))


In [75]:
validate_udf(df['Date'])


TypeError: 'Column' object is not callable

In [79]:
#df.show()

In [80]:
#df.select(col('Date'), to_date(col('Date'), format='dd/MM/yy').alias('new')).show()

In [56]:
#invalid_df = df.filter(~validate_udf(col('Date')))
invalid_df = df.filter(~validate_udf(col('Date')).cast("boolean"))

In [81]:
#invalid_df.show()

In [69]:
#validate_udf("20/12/23").head()[0]
validate_udf("20/12/23").cast("array<boolean>").head()[0]

TypeError: 'Column' object is not callable

In [70]:
validate_udf("20/12/23").alias("validation_result").select("validation_result").show()

TypeError: 'Column' object is not callable

In [ ]:
from pyspark.sql.types import StructType, BooleanType

data = [("20/12/23",)]  # Tuple with your test string
schema = StructType([StructField("date_str", StringType(), True)])
temp_df = spark.createDataFrame(data, schema)


### UDF handling test

In [83]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName('SparkUDF').getOrCreate()

columns = ["Date"]
data = [("20/12/23"),
    ("21/12/23"),
    ("20/11/23")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

TypeError: Can not infer schema for type: <class 'str'>

In [85]:
from pyspark.sql.types import StructType, BooleanType, StructField

data = [("20/12/23",)]  # Tuple with your test string
schema = StructType([StructField("date_str", StringType(), True)])
temp_df = spark.createDataFrame(data, schema)


In [86]:
temp_df.show()

Py4JJavaError: An error occurred while calling o488.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 59.0 failed 1 times, most recent failure: Lost task 0.0 in stage 59.0 (TID 100) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrRea

### Date testing 

In [113]:

from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
               .appName('SparkByExamples.com') \
               .getOrCreate()

from pyspark.sql.functions import *

df=spark.createDataFrame([["02-03-2013"],["05-06-2023"]],["input"])
df.select(col("input"),to_date(col("input"),"MM-dd-yyyy").alias("date")) \
  .show()

#SQL
spark.sql("select to_date('02-03-2013','MM-dd-yyyy') date").show()


Py4JJavaError: An error occurred while calling o707.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 74.0 failed 1 times, most recent failure: Lost task 0.0 in stage 74.0 (TID 130) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrRea